In [11]:
from sympy.logic.boolalg import to_cnf
from sympy.abc import A, B, C, D, E, F, G, H


to_cnf((A & B) | (C & D) | (E & F))
# to_cnf((A & B) | (C & D) | (E & F) | (G & H))
# to_cnf((A & B & C) | (D & E & F))

(A | C | E) & (A | C | F) & (A | D | E) & (A | D | F) & (B | C | E) & (B | C | F) & (B | D | E) & (B | D | F)

In [ ]:
from sympy import *

In [ ]:
x, y = symbols('x, y')

In [ ]:
x, y

(x, y)

In [ ]:
x | y

x | y

In [ ]:
form = to_cnf(~(x | y) | x)

In [ ]:
form

(x | ~x) & (x | ~y)

In [ ]:
w1, w2, w3, x1, x2, x3 = symbols('w1, w2, w3, x1, x2, x3')

In [ ]:
to_cnf(w1 & x1 | x2 | w3)

w3 | x1 | x2

In [ ]:
!pip install python-sat

     |████████████████████████████████| 1.8 MB 5.1 MB/s 


In [ ]:
from pysat.examples.fm import FM
from pysat.formula import WCNF
import numpy as np

W = np.array([[0,1,0.7,0,0.2],
              [1,0,4,-7,-5],
              [0.7,4,0,100,0],
              [0,-7,100,0,-3],
              [0.2,-5,0,-3,0]])


print(W)

N = W.shape[0]
print("N =",N)
logN = np.int(np.log2(N)) + 1
print("log(N) =",logN)

def b(i,j):
    return i*logN + j + 1

def S(i,j):
    return b(N-1,logN-1) + i*N + j + 1

for i in range(N):
    for j in range(logN):
        print("b({},{}) is encoded in {}".format(i,j,b(i,j)))

for i in range(N):
    for j in range(N):
        print("T({},{}) is encoded in {}".format(i,j,S(i,j)))

wcnf = WCNF()
for i in range(N):
    for j in range(N):
        for k in range(logN):
            '''Adding the clauses S_ij -> (b_i^k -> b_j^k)'''
            wcnf.append([-S(i,j),-b(i,k),b(j,k)])
            '''Adding the clauses S_ij -> (b_j^k -> b_i^k)'''
            wcnf.append([-S(i,j),-b(j,k),b(i,k)])

        '''Building the cnf for AND_{k=1}^logN (b_i^k <-> b_j^k) -> S_ij
        Notice that the cnf of the formula 
        (a <-> b) and (c <-> d) -> e is the following set of clauses
        a or b or c or d or e 
        -a or -b or c or d or e 
        a or b or -c or -d or e 
        -a or -b or -c or -d or e '''
        bik_iff_bjk_imp_sij = [[S(i,j)]]
        for k in range(logN):
            bik_iff_bjk_imp_sij = [c + [b(i,k),b(j,k)] for c in bik_iff_bjk_imp_sij] + \
                 [c + [-b(i, k), -b(j, k)] for c in bik_iff_bjk_imp_sij]
        for c in bik_iff_bjk_imp_sij:
            wcnf.append(c)
        if W[i,j] != 0:
            wcnf.append([int(np.sign(W[i,j]))*S(i,j)],weight=np.abs(W[i,j]))

fm = FM(wcnf,verbose=10)
fm.compute()
m = [n for n in fm.model]
for i in range(N):
    print("item {} is classified in cluster {}".format(i,"".join([str(int(m[b(i,j)-1]>0)) for j in range(logN)])))


[[  0.    1.    0.7   0.    0.2]
 [  1.    0.    4.   -7.   -5. ]
 [  0.7   4.    0.  100.    0. ]
 [  0.   -7.  100.    0.   -3. ]
 [  0.2  -5.    0.   -3.    0. ]]
N = 5
log(N) = 3
b(0,0) is encoded in 1
b(0,1) is encoded in 2
b(0,2) is encoded in 3
b(1,0) is encoded in 4
b(1,1) is encoded in 5
b(1,2) is encoded in 6
b(2,0) is encoded in 7
b(2,1) is encoded in 8
b(2,2) is encoded in 9
b(3,0) is encoded in 10
b(3,1) is encoded in 11
b(3,2) is encoded in 12
b(4,0) is encoded in 13
b(4,1) is encoded in 14
b(4,2) is encoded in 15
T(0,0) is encoded in 16
T(0,1) is encoded in 17
T(0,2) is encoded in 18
T(0,3) is encoded in 19
T(0,4) is encoded in 20
T(1,0) is encoded in 21
T(1,1) is encoded in 22
T(1,2) is encoded in 23
T(1,3) is encoded in 24
T(1,4) is encoded in 25
T(2,0) is encoded in 26
T(2,1) is encoded in 27
T(2,2) is encoded in 28
T(2,3) is encoded in 29
T(2,4) is encoded in 30
T(3,0) is encoded in 31
T(3,1) is encoded in 32
T(3,2) is encoded in 33
T(3,3) is encoded in 34
T(3,4) is 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  app.launch_new_instance()
